# Workflow
1. Get pictures
2. Get features
3. Do matching
4. Get model via RANSAC
5. Compensations (improvements, a-la gain compensation etc)
6. Stitching

In [1]:
import cv2
print(f'opencv version: {cv2.__version__}')

import numpy as np
np.seterr(all='raise')

import itertools
import random
from math import log
import glob
import os.path as osp
import os

opencv version: 4.2.0


## 1. Get pictures
1. get 
2. convert to gray

In [2]:
def get_pictures_video_framing():
    pass

def get_pictures_path(path_source, ext_pattern = "*.jpeg"):
    images = glob.glob(osp.join(path_source, ext_pattern))
    images.sort()
    return images

def imread_img_gray(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img, gray

### Test

In [ ]:
imgs = get_pictures_path("pictures_to_stitch_2",)
img0, gray0 = imread_img_gray(imgs[0])
img1, gray1 = imread_img_gray(imgs[1])
# print(imgs)
# print('img',img, 'gray',gray, sep="\n")

## 2. Get features
1. create detectors
2. get key points & descriptors

In [3]:
def get_kp_des(gray, detector):
    kp, des = detector.detectAndCompute(gray, None)
    return kp, des

### Test

In [ ]:
sift = cv2.xfeatures2d.SIFT_create(500) # limit feature points to 500
surf = cv2.xfeatures2d.SURF_create() # Hessian threshold is default
orb = cv2.ORB_create(nfeatures=500) # limit feature points to 500

detector = sift
kp_0, des_0 = get_kp_des(gray0, detector)
kp_1, des_1 = get_kp_des(gray1, detector)
# print('kp_0',kp_0, 'des_0',des_0, sep="\n")
# print('kp_1',kp_1, 'des_1',des_1, sep="\n")

In [ ]:
print(kp_0[0].convert.__doc__)

In [ ]:
p_kp_0 = kp_0[0].convert(kp_0)
len(p_kp_0)

In [ ]:
des_0[0].__dir__()

In [ ]:
print(des_0[0].compress.__doc__)

## 3. Do matching
1. Lowe criteria (ratio of the distance to the two closest matching features)
2. two-sided matching
3. prepare for RANSAC format

In [4]:
def match(des_0, des_1, dist_ratio = 0.7):
    """ 
    For each descriptor in the first image, select its match in the second image.
    
    Input: 
        "des_0" - descriptors for the first image
        "des_1" - same for second image 
        "dist_ratio" - empiric coefficient for Lowe criteria
    
    Return: 
        "matchscores" - list of indexes for points from the 2nd picture for every position (kp number) in first
        e.g: matchscores[0] = 1 means that for the first key point in the 1st image the closest is 
                                the second key point in the 2nd image
    """
    
#     assert des_0.shape[0]==des_1.shape[0], "Error: des_0 & des_1 has different sizes"
    
    des_0_size = des_0.shape[0]
    
    matchscores = [-1] * des_0_size
    
    for i, p in enumerate(des_0):
        distances = list(map(lambda x: np.linalg.norm(x - p), des_1))
        idx = np.argsort(distances)
        if distances[idx[0]] < dist_ratio * distances[idx[1]]:
            matchscores[i] = idx[0]
    return matchscores

In [5]:
def match_twosided(des_0,des_1):   
    """ Two-sided symmetric version of match() - see above """
    
    matches_1_to_2 = np.array(match(des_0, des_1))
    matches_2_to_1 = np.array(match(des_1, des_0))
    
    # remove matches that are not symmetric
    for i, n in enumerate(matches_1_to_2):
        if n == -1:
            continue
        if matches_2_to_1[n] != i:
            matches_1_to_2[i] = -1
    
    return matches_1_to_2

In [6]:
def matches_to_mutual_corr_list(matches_1_to_2):
    ''' Adapt for RANSAC fuction format: [# from 1st, # from 2nd image]'''
    mutual_corr_list = []
    for indx, match in enumerate(matches_1_to_2):
        if match != -1:
            mutual_corr_list.append([indx, match])  #[key point from 1st image, same from 2nd]
    return mutual_corr_list

### Test

In [ ]:
matches_1_to_2 = match_twosided(des_0,des_1)
# print(matches_1_to_2)
mutual_corr_list = matches_to_mutual_corr_list(matches_1_to_2)
print(mutual_corr_list)

## 4. Get model via RANSAC
1. sampling
2. determine error
3. iterations required
4. determine inliers
5. find homography

http://www.cse.yorku.ca/~kosta/CompVis_Notes/ransac.pdf

In [16]:
def get_keypoints_pt_by_sample(kp_0,kp_1,sample):
    '''
    Given list of points, return list of their coordinates to calculate homography matrix
    '''
    kp_0_sample=[]
    kp_1_sample=[]
    for i,j in sample:
        kp_0_sample.append(kp_0[i].pt)
        kp_1_sample.append(kp_1[j].pt)
    return kp_0_sample, kp_1_sample

def if_2_vectors_collinear(vec1,vec2,eps = 1e-6):
    if_collinear = False
    vec1_norm = np.linalg.norm(vec1)
    vec2_norm = np.linalg.norm(vec2)
    if vec1_norm>eps and vec2_norm>eps: 
        vec_dot = np.dot(vec1,vec2)/vec1_norm/vec2_norm
        if abs(vec_dot-1)<=eps:
            if_collinear = True
    return if_collinear

def if_3collinear_in_set(sample_set, kp, eps = 1e-6):
    '''
    check if any 3 points in a set (e.g. 4 points for homograpy) is collinear
    which is inappropriate to find homography matrix
    
    Through dot product for 2 dimensional.
    p.s.: cos(0 grad) = 1 :)
    eps (by def) = 10^(-6)
    
    Input:
        'sample_set' - 4 points from 1 image to find homography matrix
        'kp' - key points from set
        'eps' - accuracy, 10^(-6) by default
    
    Return:
        'if_collinear' - boolean if there is a 3 point on one line
    '''
    
    # get points from kp
    points = []
    for point_num in sample_set:
        points.append(kp[point_num])
    
    # for each combination of 3 points find collinear
    if_collinear = False
    for three_points in itertools.combinations(points,3):
        vec1 = np.array(three_points[1].pt)-np.array(three_points[0].pt)
        vec2 = np.array(three_points[2].pt)-np.array(three_points[0].pt)
        
        if_collinear = if_2_vectors_collinear(vec1,vec2,eps)
        if if_collinear:
            break
    return if_collinear

def get_4_different_pairs(mutual_corr_list, kp_0, kp_1):
    sample_set_1 = set()
    sample_set_2 = set()
    
    while True:
        sample = random.sample(mutual_corr_list,4)
        sample_set_1.clear()
        sample_set_2.clear()
        for i,j in sample:
            sample_set_1.add(i)
            sample_set_2.add(j)
        if len(sample_set_1) ==4 and len(sample_set_2)==4: 
            if not if_3collinear_in_set(sample_set_1, kp_0) and not if_3collinear_in_set(sample_set_2, kp_1):
                break

    return sample

In [8]:
def dist_by_angle(vec_0,vec_1): 
    ''' 
    If we use distance as angle between 2 vectors.
    But it's not a common metrics
    '''

    vec_0_norm = vec_0/np.linalg.norm(vec_0)
    vec_1_norm = vec_1/np.linalg.norm(vec_1)
    
    cross_prod = np.dot(vec_0_norm,vec_1_norm.T)
    
    distance_angle = np.arccos(cross_prod)
    
    return distance_angle

In [9]:
def get_ransac_req_iter_num(e=0.5,s=4,p=0.99):
    """
    [N] - max number of iterations is determined to ensure that the probability (p),
    that at least one of the sets of random selected samples doesn't include outlier. 
    
    [1-p = (1-(1-e)^m)^N] -> [N = log(1-p)/log(1-(1-e)^s)]
    
    [e] - probability that a point is an outlier
    [s] - minimum required number of points in the sample: 4 pairs is for computing homography (1 pair is one point)
    [p] - desired probability that at least one of the sets of random selected samples doesn't include outlier
    
    """
    
    iter_num_required = log(1-p)/log(1-(1-e)**s)
    req_iter = int(iter_num_required)+1
    return req_iter

In [10]:
def get_inliers(h, sample, kp_0, kp_1, mutual_corr_list, threshold, dist="norm"):
    '''
    calculate inliers for all points from mutual_corr_list
    using 
    '''
    appropriate = []
    total_distance = 0
    for i,j in mutual_corr_list:
        kp_1_h = h.dot(np.array((kp_1[j].pt[0], kp_1[j].pt[1], 1), dtype = "float64"))
        kp_1_h /= kp_1_h[2]
        kp_0_orig = np.array((kp_0[i].pt[0], kp_0[i].pt[1], 1), dtype = "float64")
        
        if dist == 'norm':
            distance = cv2.norm(kp_1_h - kp_0_orig)
        else:
            distance = dist_by_angle(kp_0_h,kp_1_orig)
        
        if distance <= threshold:
            appropriate.append(distance)
            
        total_distance += distance
        
    return len(appropriate), total_distance

In [17]:
def ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list,threshold, iter_num_required=0,
                                  dist = 'norm'):
    '''
    0) calculate required number of iterations
    repeat for required number of iterations:
    1) get sample of kps
    2) calculate Homography matrix
    3) calculate inliers & total error
    4) save the matrix with higher number of matchinliers & lower total error
    '''
    # 0
    if iter_num_required == 0:
        iter_num_required = get_ransac_req_iter_num()
    # print(kp_0)
    # kp_0 = np.array([kp_0[i].pt for i, _ in mutual_corr_list])
    # kp_1 = np.array([kp_1[i].pt for _, i in mutual_corr_list])
    # best_homography, _ = cv2.findHomography(kp_1, kp_0, cv2.RANSAC, threshold)

    best_homography = []
    max_inliers_num = -1 
    total_error_min = -1
    for x in range(iter_num_required):
        # 1
        sample = get_4_different_pairs(mutual_corr_list, kp_0, kp_1)
        kp_0_sample, kp_1_sample = get_keypoints_pt_by_sample(kp_0,kp_1,sample)
        # 2
        # get matrix h to warp right image to the left: kp_1 - source, kp_0 - destination
        h, status = cv2.findHomography(np.array(kp_1_sample), np.array(kp_0_sample), 10**6)
        # 3
        curr_inliers_num, total_error = get_inliers(h,sample,kp_0,kp_1,mutual_corr_list,threshold, dist)
        
        if_equal_inliers_less_error = max_inliers_num == curr_inliers_num \
                                      and total_error_min!=-1 and total_error_min>total_error
        if max_inliers_num < curr_inliers_num or if_equal_inliers_less_error:
#             print(f'Change of model!!!', 
#                   f'Previous max_inliers_num: {max_inliers_num}',
#                   f'Previous total_error_min: {total_error_min}',
#                   f'Current curr_inliers_num: {curr_inliers_num}',
#                   f'Current total_error: {total_error}',
#                   sep = '\n')
            max_inliers_num = curr_inliers_num
            total_error_min = total_error
            best_homography = h
        
    print(f'Number of iterations: {iter_num_required}',
          f'Maximum inliers: {max_inliers_num}',
          f'Total error min: {total_error_min}', 
          f'Best homography: {best_homography}', 
          sep = '\n')
    return max_inliers_num, total_error_min, best_homography

### Test

In [ ]:
threshold = 4
inliers, error, h = ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list, 
                                                  threshold, iter_num_required=0, dist = "norm")

## 5. Compensations (improvements, a-la gain compensation etc)

In [ ]:
pass

## 6. Stitching

In [13]:
# matrix h is to warp right image to the left: kp_1 - source, kp_0 - destination
def warp_n_stitch_imgs(h, img0, img1): #, path_results, path_source):
    if np.all(h) != 0: 
        img_1_after_homography = img1.copy()
        img_0_ext = cv2.copyMakeBorder(img0, 
                                       0, 0, 0, img_1_after_homography.shape[1], 
                                       cv2.BORDER_CONSTANT, (0,0,0))
        result_img_length = img0.shape[1]+img1.shape[1]
        result_img_height = img1.shape[0]
        result_img = cv2.warpPerspective(img_1_after_homography, h, 
                                         (result_img_length, result_img_height), 
                                         img_0_ext,
                                         borderMode = cv2.BORDER_TRANSPARENT)
#         result_img[:img0.shape[0], :img0.shape[1]] = img0
#         result_imwrite = cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_sift.jpg'),result_img)
        return result_img
    else: 
        return False

### Test

In [ ]:
path_results = "results"
path_source = "pictures_to_stitch_2"
img_1_after_homography = img1.copy()

In [ ]:
img_0_ext = cv2.copyMakeBorder(img0,0,0,0,img0.shape[1],cv2.BORDER_CONSTANT)

In [ ]:
result_img_length = img0.shape[1]*2
result_img_height = img1.shape[0]
result_img = cv2.warpPerspective(img_1_after_homography, h, 
                                 (result_img_length,result_img_height),
                                 img_0_ext,
                                 borderMode=cv2.BORDER_TRANSPARENT)
result_imwrite = cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_sift_transparent.jpg'),result_img)

# Main

In [ ]:
path_source = "pictures_to_stitch_2"

imgs = get_pictures_path(path_source)
img0, gray0 = imread_img_gray(imgs[0])
img1, gray1 = imread_img_gray(imgs[1])
sift = cv2.xfeatures2d.SIFT_create(500) # limit feature points to 500
surf = cv2.xfeatures2d.SURF_create() # Hessian threshold is default
orb = cv2.ORB_create(nfeatures=500) # limit feature points to 500
detector = sift
kp_0, des_0 = get_kp_des(gray0, detector)
kp_1, des_1 = get_kp_des(gray1, detector)
matches_1_to_2 = match_twosided(des_0,des_1)
# print(matches_1_to_2)
mutual_corr_list = matches_to_mutual_corr_list(matches_1_to_2)
# print(mutual_corr_list)
threshold = 4
inliers, error, h = ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list, 
                                                  threshold, iter_num_required=0, dist = "norm")
result_img = warp_n_stitch_imgs(h, img0, img1)

path_results = "results"

result_imwrite = cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_sift.jpg'),result_img)

print(result_imwrite)

In [14]:
def consecutive_stitching(path_to_imgs, detector, threshold=4):
    imgs = get_pictures_path(path_to_imgs)
    h_total = np.array([0,])
    for i in range(len(imgs)-1):
        img0, gray0 = imread_img_gray(imgs[i])
        img1, gray1 = imread_img_gray(imgs[i+1])
        kp_0, des_0 = get_kp_des(gray0, detector)
        kp_1, des_1 = get_kp_des(gray1, detector)
        matches_1_to_2 = match_twosided(des_0,des_1)
        mutual_corr_list = matches_to_mutual_corr_list(matches_1_to_2)
        inliers, error, h = ransac_find_homography_matrix(kp_0, kp_1, mutual_corr_list, 
                                                  threshold, iter_num_required=0, dist = "norm")
        if h_total.all() ==0:
            h_total = h
            result_img = warp_n_stitch_imgs(h_total, img0, img1)
        else:
            h_total = h.dot(h_total)
            result_img = warp_n_stitch_imgs(h_total, result_img, img1)          
    return result_img

In [18]:
path_source = "pictures_to_stitch_2"
path_results = "results"

sift = cv2.xfeatures2d.SIFT_create(500) # limit feature points to 500
surf = cv2.xfeatures2d.SURF_create() # Hessian threshold is default
orb = cv2.ORB_create(nfeatures=500) # limit feature points to 500

result_img_consecutive = consecutive_stitching(path_source, sift, 4)

result_imwrite = cv2.imwrite(os.path.join(path_results,'warped',f'{path_source}_total_sift.jpg'),result_img_consecutive)

Number of iterations: 72
Maximum inliers: 38
Total error min: 1001.8419337088656
Best homography: [[ 8.05714439e-01 -1.85982517e-02  3.81392970e+02]
 [-2.00330303e-01  9.54930764e-01  2.64136549e+01]
 [-3.27608432e-04 -2.44878327e-06  1.00000000e+00]]
Number of iterations: 72
Maximum inliers: 42
Total error min: 2425.222740073378
Best homography: [[ 8.30332685e-01 -2.13268199e-02  3.93199490e+02]
 [-2.28754179e-01  9.70691934e-01 -2.31938765e+00]
 [-3.78617581e-04  1.08082920e-05  1.00000000e+00]]
